In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
plt.style.use("ggplot")
%matplotlib inline

from transform import get_X_y
%load_ext autoreload
%autoreload 2

In [2]:
# import tensorflow modules. tensorflow was run on a docker container

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import Precision, Recall

# tf.config.threading.set_intra_op_parallelism_threads(12)

In [3]:
## import sklearn modules

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

In [4]:
orbit = pd.read_csv('../data/samples_2_bodies_3_dim_1_m_com.csv')

In [5]:
orbit

,sim_id,m_1,m_2,rx_1_0,ry_1_0,rz_1_0,rx_2_0,ry_2_0,rz_2_0,vx_1_0,...,rz_1_1000,rx_2_1000,ry_2_1000,rz_2_1000,vx_1_1000,vy_1_1000,vz_1_1000,vx_2_1000,vy_2_1000,vz_2_1000
0,0.0,1.0,1.0,0.072812,0.339681,0.234807,-0.072812,-0.339681,-0.234807,0.032668,...,0.245414,-0.062839,-0.156803,-0.245414,-0.104210,-0.721673,-0.262195,0.104210,0.721673,0.262195
1,0.0,1.0,1.0,0.054407,0.105241,0.222055,-0.054407,-0.105241,-0.222055,-0.151459,...,0.001457,-0.017101,-0.250963,-0.001457,0.167632,0.763737,0.546329,-0.167632,-0.763737,-0.546329
2,0.0,1.0,1.0,0.015466,-0.034834,0.083430,-0.015466,0.034834,-0.083430,-0.452646,...,0.080874,-0.039404,-0.331130,-0.080874,0.128510,0.343728,0.494653,-0.128510,-0.343728,-0.494653
3,0.0,1.0,1.0,0.060140,0.364899,0.167492,-0.060140,-0.364899,-0.167492,0.083064,...,0.261195,-0.074577,-0.281935,-0.261195,-0.013481,-0.439943,0.074785,0.013481,0.439943,-0.074785
4,0.0,1.0,1.0,0.017181,0.251330,0.001720,-0.017181,-0.251330,-0.001720,0.167503,...,0.181902,-0.063202,-0.364724,-0.181902,0.074315,-0.030131,0.357845,-0.074315,0.030131,-0.357845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,1999.0,1.0,1.0,-0.056611,0.192364,-0.247187,0.056611,-0.192364,0.247187,-0.214536,...,-0.125396,0.077384,-0.004295,0.125396,0.130841,-1.165302,1.163385,-0.130841,1.165302,-1.163385
199996,1999.0,1.0,1.0,-0.062381,-0.046865,-0.059740,0.062381,0.046865,0.059740,0.604474,...,0.126458,-0.132333,-0.099748,-0.126458,0.632941,1.154271,0.048515,-0.632941,-1.154271,-0.048515
199997,1999.0,1.0,1.0,0.250455,0.441298,0.031888,-0.250455,-0.441298,-0.031888,0.071420,...,-0.021397,-0.256480,-0.517709,0.021397,-0.002030,0.345303,-0.286874,0.002030,-0.345303,0.286874
199998,1999.0,1.0,1.0,0.117538,0.560847,-0.275648,-0.117538,-0.560847,0.275648,-0.211689,...,-0.294678,-0.076410,-0.505170,0.294678,-0.231047,-0.354711,-0.072457,0.231047,0.354711,0.072457


In [6]:
steps = 10
X, y = get_X_y(orbit, steps, False, True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
# create and compile neural network

n_samples, n_feats = X_train.shape
_, n_outputs = y_train.shape

model = Sequential()

hidden_units = n_feats
epochs=100
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8

scale = 10


hidden_layer = Dense(units=hidden_units,
                input_dim=n_feats,
                kernel_initializer='uniform',
                activation='relu')

hidden_layer2 = Dense(units=hidden_units*scale,
                input_dim=hidden_units,
                kernel_initializer='uniform',
                activation='relu')

hidden_layer3 = Dense(units=hidden_units*scale,
                input_dim=hidden_units*scale,
                kernel_initializer='uniform',
                activation='relu')

hidden_layer4 = Dense(units=n_outputs,
                input_dim=hidden_units*scale,
                kernel_initializer='uniform',
                activation='relu')

output_layer = Dense(units=n_outputs,
                input_dim=hidden_units,
                kernel_initializer='uniform',
                activation='linear')

model.add(hidden_layer)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer2)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer3)
# model.add(Dropout(rate=0.1))
# model.add(hidden_layer4)
# model.add(Dropout(rate=0.1))
model.add(output_layer)
model.add(Dropout(rate=0.1))

sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
# sgd = SGD(lr=learning_rate)
model.compile(loss='mean_squared_error', 
              optimizer=sgd)

In [8]:
history = model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=5000, 
          verbose=2, 
          validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 - 0s - loss: 5.9719 - val_loss: 0.7138
Epoch 2/100
120000/120000 - 0s - loss: 1.5788 - val_loss: 0.4609
Epoch 3/100
120000/120000 - 0s - loss: 1.5467 - val_loss: 0.4581
Epoch 4/100
120000/120000 - 0s - loss: 1.5249 - val_loss: 0.4591
Epoch 5/100
120000/120000 - 0s - loss: 1.4900 - val_loss: 0.4590
Epoch 6/100
120000/120000 - 0s - loss: 1.5189 - val_loss: 0.4590
Epoch 7/100
120000/120000 - 0s - loss: 1.5259 - val_loss: 0.4592
Epoch 8/100
120000/120000 - 0s - loss: 1.5154 - val_loss: 0.4590
Epoch 9/100
120000/120000 - 0s - loss: 1.5337 - val_loss: 0.4589
Epoch 10/100
120000/120000 - 0s - loss: 1.5507 - val_loss: 0.4593
Epoch 11/100
120000/120000 - 0s - loss: 1.5335 - val_loss: 0.4591
Epoch 12/100
120000/120000 - 0s - loss: 1.5177 - val_loss: 0.4590
Epoch 13/100
120000/120000 - 0s - loss: 1.5141 - val_loss: 0.4591
Epoch 14/100
120000/120000 - 0s - loss: 1.5308 - val_loss: 0.4589
Epoch 15/100
120000/120000 - 0s - 

In [17]:
# fig, ax = plt.subplots(figsize=(12,8))
# ax.set_title('Loss')
# ax.plot(np.sqrt(history.history['loss']), label='train')
# ax.plot(np.sqrt(history.history['val_loss']), label='test')
# ax.legend();
y_pred = model.predict(X_test)
print(steps, np.sqrt(mean_squared_error(y_test, y_pred)))

10 0.6736981971991173


In [18]:
y_test[0], y_pred[0]

(array([ 0.26608697, -0.02349626,  0.27246018, -0.15814119, -0.41994822,
         1.49872771]),
 array([ 0.263269  , -0.0228501 ,  0.3190683 ,  0.00988263,  0.00106928,
         0.01340782], dtype=float32))

In [19]:
steps = 100
X, y = get_X_y(orbit, steps, False, True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
history = model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=5000, 
          verbose=2, 
          validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 - 0s - loss: 1.6800 - val_loss: 0.5162
Epoch 2/100
120000/120000 - 0s - loss: 1.6566 - val_loss: 0.5112
Epoch 3/100
120000/120000 - 0s - loss: 1.6425 - val_loss: 0.5086
Epoch 4/100
120000/120000 - 0s - loss: 1.6114 - val_loss: 0.5095
Epoch 5/100
120000/120000 - 0s - loss: 1.6326 - val_loss: 0.5087
Epoch 6/100
120000/120000 - 0s - loss: 1.6343 - val_loss: 0.5083
Epoch 7/100
120000/120000 - 0s - loss: 1.6398 - val_loss: 0.5085
Epoch 8/100
120000/120000 - 0s - loss: 1.6354 - val_loss: 0.5086
Epoch 9/100
120000/120000 - 0s - loss: 1.6287 - val_loss: 0.5074
Epoch 10/100
120000/120000 - 0s - loss: 1.6453 - val_loss: 0.5076
Epoch 11/100
120000/120000 - 0s - loss: 1.6891 - val_loss: 0.5079
Epoch 12/100
120000/120000 - 0s - loss: 1.6157 - val_loss: 0.5081
Epoch 13/100
120000/120000 - 0s - loss: 1.5981 - val_loss: 0.5078
Epoch 14/100
120000/120000 - 0s - loss: 1.6531 - val_loss: 0.5077
Epoch 15/100
120000/120000 - 0s - 

In [21]:
# fig, ax = plt.subplots(figsize=(12,8))
# ax.set_title('Loss')
# ax.plot(np.sqrt(history.history['loss']), label='train')
# ax.plot(np.sqrt(history.history['val_loss']), label='test')
# ax.legend();
y_pred = model.predict(X_test)
print(steps, np.sqrt(mean_squared_error(y_test, y_pred)))

100 0.7087378867937281


In [22]:
y_test[0], y_pred[0]

(array([-0.12971355, -0.43622705, -0.80293813,  1.16449215,  0.67082781,
         0.28566427]),
 array([-0.48914886, -0.41010717, -0.6369747 , -0.02621982, -0.0166013 ,
        -0.038638  ], dtype=float32))

In [23]:
steps = 1000
X, y = get_X_y(orbit, steps, False, True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [24]:
history = model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=5000, 
          verbose=2, 
          validation_split=0.2)

Train on 120000 samples, validate on 30000 samples
Epoch 1/100
120000/120000 - 0s - loss: 3.4481 - val_loss: 1.5957
Epoch 2/100
120000/120000 - 0s - loss: 3.4048 - val_loss: 1.5804
Epoch 3/100
120000/120000 - 0s - loss: 3.4237 - val_loss: 1.5698
Epoch 4/100
120000/120000 - 0s - loss: 3.3607 - val_loss: 1.5748
Epoch 5/100
120000/120000 - 0s - loss: 3.4361 - val_loss: 1.5688
Epoch 6/100
120000/120000 - 0s - loss: 3.3422 - val_loss: 1.5624
Epoch 7/100
120000/120000 - 0s - loss: 3.3235 - val_loss: 1.5588
Epoch 8/100
120000/120000 - 0s - loss: 3.3703 - val_loss: 1.5505
Epoch 9/100
120000/120000 - 0s - loss: 3.3535 - val_loss: 1.5486
Epoch 10/100
120000/120000 - 0s - loss: 3.3587 - val_loss: 1.5550
Epoch 11/100
120000/120000 - 0s - loss: 3.3690 - val_loss: 1.5562
Epoch 12/100
120000/120000 - 0s - loss: 3.2912 - val_loss: 1.5562
Epoch 13/100
120000/120000 - 0s - loss: 3.4068 - val_loss: 1.5449
Epoch 14/100
120000/120000 - 0s - loss: 3.3231 - val_loss: 1.5434
Epoch 15/100
120000/120000 - 0s - 

In [25]:
# fig, ax = plt.subplots(figsize=(12,8))
# ax.set_title('Loss')
# ax.plot(np.sqrt(history.history['loss']), label='train')
# ax.plot(np.sqrt(history.history['val_loss']), label='test')
# ax.legend();
y_pred = model.predict(X_test)
print(steps, np.sqrt(mean_squared_error(y_test, y_pred)))

1000 1.2318787586011237


In [26]:
y_test[0], y_pred[0]

(array([-0.14316453, -0.40070764,  0.48438827,  0.46492697, -0.26274416,
         0.98961032]),
 array([-0.68804413, -1.3492872 ,  1.3039368 , -0.02815423, -0.05219495,
         0.05298544], dtype=float32))